<a href="https://colab.research.google.com/github/SilvioIA2025/ALura2025/blob/main/Projeto_de_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai
client = genai.Client()
from IPython.display import HTML, Markdown

MODEL_ID = "gemini-2.5-flash-preview-04-17"

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config = {"tools": [{"google_search":{}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A Imersão IA com Google Gemini da Alura aconteceu entre os dias 12 e 16 de maio de 2025. As inscrições estiveram abertas até o dia 11 de maio de 2025.

Esta edição foi online e gratuita, e abordou temas como a criação de prompts eficazes, automação de tarefas e desenvolvimento de agentes autônomos utilizando o Google Gemini.

Não foram encontradas informações sobre a data de uma próxima edição da Imersão IA com Google Gemini da Alura nas buscas realizadas.

In [4]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima Imersão IA com Google Gemini Alura data', 'Alura Imersão IA Google Gemini próximas edições']
Páginas utilizadas na resposta: starten.tech, rodrigostoledo.com, thallesbenicio.com.br, alura.com.br



In [5]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [6]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [16]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [106]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [222]:
##################################################################
# --- Agente 1: Buscador de Custos de Viagens Internacionais --- #
##################################################################
def agente_buscador(topico, pais, perfil, data_de_inicio, data_de_fim, numero_adultos, numero_crianças):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Como um especialista em calcular gastos em viagens internacionais, meu objetivo é fornecer uma estimativa detalhada e otimizada dos seus custos,
        considerando suas preferências e buscando as melhores opções em cada categoria.

        Instruções para o Especialista em Cálculos de Viagem:

        1. Compreensão do Perfil do Viajante:

        Destino: País(es) e Cidade(s).
        Período: Datas de partida e retorno (para considerar alta/baixa temporada e eventos).
        Número de Viajantes: Adultos e crianças.
        Estilo de Viagem: (e.g., mochileiro, econômico, confortável, luxuoso) para calibrar os valores.
        Interesses Específicos: (e.g., gastronomia, museus, aventura, compras) para alocar orçamento.

        2. Análise e Estimativa por Categoria:

        Voos (Melhores Opções):
        Pesquisar e comparar companhias aéreas, escalas, duração do voo e aeroportos de partida/chegada.
        Considerar voos diretos vs. com escalas (custo/benefício).
        Avaliar épocas de compra (antecedência) e flexibilidade de datas para melhores preços.
        Incluir custos de bagagem despachada, se aplicável.

        Aluguel de Carro:
        Avaliar a necessidade do aluguel versus transporte público no destino.
        Selecionar o tipo de veículo adequado (econômico, SUV, etc.).
        Incluir custos de seguro obrigatório e opcionais, combustível, pedágios e estacionamento.
        Comparar locadoras e políticas de retirada/devolução.

        Hotéis/Hospedagem (Melhores Opções):
        Sugerir tipos de hospedagem (hotéis, hostels, Airbnb, pousadas) com base no estilo de viagem.
        Considerar localização (proximidade a atrações, transporte), avaliações, comodidades (café da manhã, Wi-Fi) e política de cancelamento.
        Estimar custo por noite, ajustado pela temporada.

        Refeições:
        Estimar um custo médio diário por pessoa para café da manhã, almoço e jantar, diferenciando entre restaurantes econômicos, médios e sofisticados.
        Considerar o custo de lanches, bebidas e gorjetas.
        Perguntar se o viajante pretende cozinhar algumas refeições.

        Atrações Turísticas e Atividades:
        Listar as principais atrações pagas e o custo de seus ingressos.
        Sugerir passes turísticos (se vantajosos para a quantidade de atrações).
        Incluir custo de tours guiados, shows ou atividades específicas desejadas.

        Transporte Local:
        Custos de metrô, ônibus, bondes, táxis ou aplicativos de transporte (Uber, Lyft, etc.).
        Estimar passes diários/semanais se forem mais econômicos.
        Despesas Gerais e Extras:

        Seguro Viagem: Item obrigatório e essencial.
        Visto e Taxas de Entrada: Se aplicável ao destino.
        Comunicações: Custo de chip local ou roaming internacional.
        Emergências/Imprevistos: Sugerir uma reserva de 10-15% do total para gastos não planejados.
        Compras/Souvenirs: Definir um orçamento para compras pessoais e lembrancinhas.
        Outros: Lavanderia, gorjetas adicionais, pequenos gastos diversos.

        3. Saída Final:
        Faça voce a escolha de uma cidade a ser visitada no pais, caso o não tenha sido escolhida, retorne todos os resultados de custo pelo
        estilo de Viagem selecionado no perfil (mochileiro, economico, confortavel e luxuoso) e voce deve escolher automaticamente dois
        Interesses Específicos: dentre Gastronomia, Museus, Aventura e Compras.
        Apresentar um orçamento detalhado por categoria (Voos, Aluguel de Carro, Hotéis/Hospedagem, Refeições, Atrações Turísticas e Atividades, Transporte Local, Seguro Viagem
        Visto e Taxas de Entrada, Comunicações, Emergências/Imprevistos, Compras/Souvenirs e Outros ) , com uma estimativa total convertida para a moeda de origem do viajante
        (geralmente BRL) e da moeda local do destino, utilizando a taxa de câmbio atualizada. Oferecer dicas de economia e personalização.

        A sua tarefa é usar a ferramenta de busca do google (google_search) respeitando a data inicial e final da viagem
        Foque em no maximo 4 opções relevantes respeitando os estilos de Viagem: (mochileiro, econômico, confortável, luxuoso), com base no preço, categoria, e avaliaçoes dos clientes anteriores.
        Se uma avaliação tiver poucas informações ou reações entusiasmadas, é possivel que ela não seja tão relevante assim
        e pode ser substituido por outro que tenha mais.
        Essas avaliações relevantes devem ser atuais, de no maximo um tres meses da data de hoje.
        """,
        description = "Agente que busca melhores voos, carros, hoteis, refeiçoes, atrações turisticas, transporte e custos diretos e indiretos de viagem indicado no Google",
        tools=[google_search],
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nPaís: {pais}\nPerfil: {perfil}\nData de inicio: {data_de_inicio}\nData de fim: {data_de_fim}\nNumero_adultos: {numero_adultos}\nNumero_crianças: {numero_crianças}"
    # Executa o agente
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados

In [223]:
##########################################
# --- Agente 2: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, lancamentos_buscados):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em documentos profissionais.
            Por ter um público amplo, entre 18 e 70 anos, use sempre um tom de escrita adequado.
            Revise o lancamentos_buscados abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Separar por uma linha em branco cada item da lista.
            Se o lancamentos_buscados estiver bom, repita apenas o texto novamente'.
            Caso haja problemas, corrija e refaça o texto de maneira adequada, gerando uma versão corrigida
            """,
        description="Agente revisor de post para documentos profissionais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {lancamentos_buscados}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [224]:
##################################################################
# --- Agente 3: Buscador da Melhor alternativa de levar ME --- #
##################################################################
def agente_moeda(topico, pais):
    imagem = Agent(
        name="agente_moeda",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Voce como especialista em cambio da Viktap, por favor me faça um breve resumo de qual a melhor maneira para eu converter o meu
        dinheiro local para o dinheiro internacional, considerando cartao de credito, cartao de debito, cartao internacional, remessa para conta
        no estrangeiro, ou PIX. No caso do PIX, fale sobre o viktap consultando a fonte no site https://viktap.com.br/
        """,
        tools=[google_search],
    )
    entrada_do_agente_moeda = f"Tópico: {topico}\nPais: {pais}"
    # Executa o agente
    escolha_moeda = call_agent(imagem, entrada_do_agente_moeda)
    return escolha_moeda

In [225]:
####################################################################
# --- Agente 4: Sugestao de Voo, pelo perfil escolhido ----------- #
####################################################################
def agente_viagem(topico, pais, perfil, aeroporto, data_inicio, data_retorno):
    viagem = Agent(
        name="agente_viagem",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Como um especialista em viagens internacionais da LATAM, faça apenas uma opção resumida para um voo consultando a LATAM saindo
        do campo aeroporto e na data de ida data_inicio e data de retorno data_retorno
        por favor escolha um voo direto de ida e um voo direto de volta a sua escolha informando o numero do voo e horario de partida e chegada
        """,
        tools=[google_search],
    )
    entrada_do_agente_viagem = f"Tópico: {topico}\nPais: {pais}\nPerfil: {perfil}\nAeroporto: {aeroporto}\nInicio: {data_inicio}\nFim: {data_retorno}"
    # Executa o agente
    viagem_pais = call_agent(viagem, entrada_do_agente_viagem)
    return viagem_pais

In [226]:
################################################################
# --- Agente 5: Sugestao de Hotel, refeição e passeio turistico
#     pelo perfil escolhido ---------------------------------- #
################################################################
def agente_outros(topico, pais, perfil, data_inicio, data_retorno):
    outros = Agent(
        name="agente_outros",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Como um especialista em viagens internacionais, faça apenas uma opção resumida para hoteis, refeição e passeios turisticos
        levando em consideração na data de ida data_inicio e data de retorno data_retorno
        por favor escolha somente 3 opções de hoteis, passeios turisticos e refeicoes.
        """,
        tools=[google_search],
    )
    entrada_do_agente_outros = f"Tópico: {topico}\nPais: {pais}\nPerfil: {perfil}\nInicio: {data_inicio}\nFim: {data_retorno}"
    # Executa o agente
    outros = call_agent(outros, entrada_do_agente_outros)
    return outros

In [227]:
##agente_buscador(topico, pais, data_inicio, data_retorno, numero_adultos, numero_crianças)
##agente_revisor(topico, lancamentos_buscados)
##agente_moeda(topico, pais)
##agente_viagem(topico, pais, perfil, aeroporto, data_inicio, data_retorno)
##agente_outros(topico, pais, perfil, data_inicio, data_retorno)

from datetime import datetime

def obter_data_do_usuario(param1):
    while True:
        if param1 == 'ida':
          data_str = input("Digite a data de inicio (DD/MM/AAAA): ")
        else:
          data_str = input("Digite a data de retorno (DD/MM/AAAA): ")

        try:
            # Tenta converter a string para um objeto datetime
            data_obj = datetime.strptime(data_str, "%d/%m/%Y")
            return data_obj
        except ValueError:
            print("Formato de data inválido. Por favor, use DD/MM/AAAA.")

def selecionar_perfil(opcoes, mensagem="Selecione uma opção:"):
    print(mensagem)
    for i, opcao in enumerate(opcoes):
        print(f"[{i + 1}] {opcao}")

    while True:
        try:
            escolha = input("Digite o número da sua escolha: ")
            indice_escolhido = int(escolha) - 1 # Ajusta para o índice da lista (0-based)

            if 0 <= indice_escolhido < len(opcoes):
                return opcoes[indice_escolhido]
            else:
                print("Número inválido. Por favor, digite um número da lista.")
        except ValueError:
            print("Entrada inválida. Por favor, digite um número.")



In [228]:
topico = 'Viagem'
pais = input("Digite o Pais/Cidade do destino: ")
data_inicio = obter_data_do_usuario('ida')
data_retorno = obter_data_do_usuario('fim')
adultos = input("Digite o numero de adultos: ")
criancas = input("Digite o numero de crianças: ")
opcoes = ["Mochileiro", "Economico", "Confortavel", "Luxuoso"]
perfil = selecionar_perfil(opcoes, mensagem="Selecione uma opção:")
print (perfil)
aeroporto = input("Digite o aeroporto de partida: ")

lancamentos_buscados = agente_buscador(topico, pais, perfil, data_inicio, data_retorno, adultos, criancas)
##print("\n Resultado do Agente 1 (Buscador)! \n")
##display(to_markdown(lancamentos_buscados))
##print("----------------------------------------------------------------")

post_final = agente_revisor(topico, lancamentos_buscados)
print("\n Recomendações para a sua viagem \n")
display(to_markdown(post_final))
print("----------------------------------------------------------------")

escolha_moeda = agente_moeda(topico, pais)
print("\n Dicas financeiras para pagar com moeda estrangeira \n")
display(to_markdown(escolha_moeda))
print("----------------------------------------------------------------")

viagem_pais = agente_viagem(topico, pais, perfil, aeroporto, data_inicio, data_retorno)
print("\n Dica de um agente de viagem da LATAM \n")
display(to_markdown(viagem_pais))
print("----------------------------------------------------------------")

outros = agente_outros(topico, pais, perfil, data_inicio, data_retorno)
print("\n Algumas dicas de Hoteis, refeiçoes e passeios turisiticos \n")
display(to_markdown(outros))
print("----------------------------------------------------------------")


Digite o Pais/Cidade do destino: Instambul
Digite a data de inicio (DD/MM/AAAA): 15/06/2025
Digite a data de retorno (DD/MM/AAAA): 30/06/2025
Digite o numero de adultos: 1
Digite o numero de crianças: 0
Selecione uma opção:
[1] Mochileiro
[2] Economico
[3] Confortavel
[4] Luxuoso
Digite o número da sua escolha: 1
Mochileiro
Digite o aeroporto de partida: GRU

 Recomendações para a sua viagem 



▌ Olá! Como seu especialista em gastos de viagem internacional, vou te ajudar a estimar os custos da sua aventura
▌ como mochileiro em Istambul.                                                                                   
▌ Com base no seu perfil e período de viagem (15/06/2025 a 30/06/2025), que compreende 15 dias na cidade, e      
▌ considerando o estilo mochileiro para 1 adulto, escolhi dois interesses específicos: Gastronomia (focando em   
▌ comida de rua e mercados locais) e Museus (priorizando opções acessíveis e culturais).                         
▌ Segue a estimativa detalhada dos gastos, otimizada para um viajante mochileiro em Istambul:                    
▌ Cotação Atual (aproximada): 1 BRL = 6.86 TRY (Lira Turca). Esta cotação pode variar. É sempre bom verificar a  
▌ taxa de câmbio no dia da viagem.                                                                               
▌ 1. Voos (Estimativa Otimizada - Ida e Volta):                                                                  
▌ Para junho de 2025, a busca por voos do Brasil para Istambul indica valores a partir de R$ 5.185. É importante 
▌ pesquisar em diferentes companhias aéreas e plataformas, considerando voos com escalas, que geralmente são mais
▌ baratos. Companhias como Turkish Airlines, Pegasus e Air Europa operam voos para Istambul.                     
▌  • Estimativa: R$ 5.500 - R$ 7.500 (Considerando variações de antecedência na compra e possíveis escalas).     
▌ 2. Aluguel de Carro:                                                                                           
▌ Para o estilo mochileiro em Istambul, o aluguel de carro não é recomendado. O transporte público na cidade é   
▌ eficiente e abrangente, além do trânsito ser intenso.                                                          
▌  • Estimativa: R$ 0                                                                                            
▌ 3. Hotéis/Hospedagem (Estimativa Otimizada - Hostel/Guesthouse):                                               
▌ Como mochileiro, hostels e guesthouses são as melhores opções pela economia e oportunidade de conhecer outros  
▌ viajantes. O custo por noite em Istambul pode variar, mas é possível encontrar opções a partir de TL 300-500.  
▌  • Estimativa para 15 noites: (300 TRY/noite * 15 noites) a (500 TRY/noite * 15 noites) = 4.500 TRY a 7.500 TRY
▌  • Convertido para BRL: (4.500 TRY / 6.86) a (7.500 TRY / 6.86) = R$ 656 a R$ 1.093                            
▌ Opções Relevantes (Pesquisar por avaliações recentes e localização):                                           
▌  • Hostels bem avaliados com boa localização em áreas como Sultanahmet ou Beyoğlu.                             
▌  • Guesthouses simples e limpas em bairros centrais.                                                           
▌ 4. Refeições (Estimativa Otimizada - Foco em Gastronomia de Rua e Econômica):                                  
▌ O interesse em gastronomia, com foco em comida de rua, é excelente para economizar em Istambul. É possível     
▌ encontrar refeições completas e saborosas por valores acessíveis. Um combo de kebab e refrigerante custa em    
▌ torno de US$ 5 (aproximadamente 170 TRY). Comida de rua como Simit e Lahmacun são ainda mais baratos.          
▌  • Estimativa diária por pessoa: Café da manhã leve (Simit, Börek, chá): 50-100 TRY; Almoço (Street food):     
▌    150-250 TRY; Jantar (Restaurante econômico ou street food reforçado): 200-350 TRY.                          
▌  • Estimativa total para 15 dias: (400 TRY/dia * 15 dias) a (700 TRY/dia * 15 dias) = 6.000 TRY a 10.500 TRY   
▌  • Convertido para BRL: (6.000 TRY / 6.86) a (10.500 TRY / 6.86) = R$ 875 a R$ 1.530                           
▌ Dica: Explore mercados locais para comprar frutas, lanches e outros itens a preços reduzidos.                  
▌ 5. Atrações Turísticas e Atividades (Estimativa Otimizada - Foco em Museus e Locais Acessíveis):               
▌ Istambul

----------------------------------------------------------------

 Dicas financeiras para pagar com moeda estrangeira 



▌ Olá! Como especialista em câmbio da Viktap, entendo sua necessidade de encontrar a melhor forma de converter   
▌ seu dinheiro para sua viagem a Istambul. Vamos analisar as opções, considerando as taxas e a praticidade em    
▌ 2025:                                                                                                          
▌ Cartão de Crédito:                                                                                             
▌  • Como funciona: Você utiliza seu cartão de crédito brasileiro para fazer compras no exterior. O valor é      
▌    convertido para reais na data de fechamento da fatura, utilizando a cotação do dólar PTAX, que geralmente é 
▌    menos favorável que o câmbio comercial.                                                                     
▌  • Custos: Em 2025, o IOF (Imposto sobre Operações Financeiras) para compras internacionais com cartão de      
▌    crédito é de 3,38%. Além disso, os bancos podem aplicar um spread (margem de lucro sobre a taxa de câmbio) e
▌    tarifas de transação internacional.                                                                         
▌  • Vantagens: Praticidade, possibilidade de parcelamento em alguns casos e programas de benefícios (milhas,    
▌    cashback) oferecidos por alguns cartões.                                                                    
▌  • Desvantagens: Custo total mais alto devido ao IOF e spread, variação cambial até o fechamento da fatura, e a
▌    necessidade de verificar se o cartão está habilitado para uso internacional.                                
▌ Cartão de Débito:                                                                                              
▌  • Cartão de Débito Brasileiro: Utilizar um cartão de débito diretamente no exterior geralmente envolve um IOF 
▌    de 3,38% sobre o valor da transação, além de possíveis taxas de saque e câmbio desfavorável.                
▌  • Cartão de Débito Internacional (Conta Global): Esta é uma das opções mais vantajosas atualmente. Você abre  
▌    uma conta em moeda estrangeira (dólar ou euro, por exemplo) e utiliza um cartão de débito vinculado a ela. A
▌    conversão do real para a moeda estrangeira ocorre no momento da recarga da conta, com IOF de 1,1%. As taxas 
▌    de câmbio geralmente são mais competitivas (câmbio comercial) e o spread é menor do que nos cartões de      
▌    crédito tradicionais. Exemplos de provedores incluem Wise, Nomad, Revolut e contas globais oferecidas por   
▌    bancos tradicionais.                                                                                        
▌  • Vantagens: IOF significativamente menor (1,1%) em comparação com o cartão de crédito, taxas de câmbio mais  
▌    favoráveis, maior controle dos gastos, e facilidade de uso para compras e saques.                           
▌  • Desvantagens: A necessidade de planejar a recarga da conta antes da viagem.                                 
▌ Cartão Internacional Pré-pago:                                                                                 
▌  • Como funciona: Similar a um cartão de débito, mas você carrega um valor em moeda estrangeira antes de usar. 
▌  • Custos: Em 2025, a alíquota de IOF é de 3,38% sobre a recarga.                                              
▌  • Vantagens: Controle de gastos por carregar apenas o valor desejado e segurança por não estar vinculado à sua
▌    conta bancária principal.                                                                                   
▌  • Desvantagens: IOF mais alto que o do cartão de débito de conta global e taxas adicionais na recarga e       
▌    saques.                                                                                                     
▌ Remessa para Conta no Estrangeiro:                                                                             
▌  • Como funciona: Enviar dinheiro de sua conta brasileira para uma conta em outro país. Pode ser para uma conta
▌    de su

----------------------------------------------------------------

 Dica de um agente de viagem da LATAM 



▌ Como especialista em viagens internacionais da LATAM, informo que a LATAM não opera voos diretos de São Paulo  
▌ (GRU) para Istambul (IST). No entanto, a LATAM possui acordo de codeshare com a Turkish Airlines, o que permite
▌ oferecer opções de voos com conexão.                                                                           
▌ Para a data de ida de 15/06/2025 e retorno em 30/06/2025, considerando a necessidade de voos com a LATAM e a   
▌ ausência de rotas diretas, uma opção seria um voo com conexão.                                                 
▌ Sugestão de voo (com conexão, operado em parceria com a Turkish Airlines):                                     
▌  • Voo de Ida (15/06/2025):                                                                                    
▌     • LATAM/Turkish Airlines (codeshare)                                                                       
▌     • Número do Voo: LA8070 / TK1594 (exemplo de codeshare)                                                    
▌     • Horário de Partida de GRU: 23:45 (aproximadamente)                                                       
▌     • Chegada em FRA (Frankfurt) para conexão.                                                                 
▌     • Partida de FRA para IST (Istambul) com Turkish Airlines.                                                 
▌     • Horário de Chegada em IST: 23:05 do dia seguinte (aproximadamente, considerando o tempo total de viagem e
▌       conexão).                                                                                                
▌  • Voo de Volta (30/06/2025):                                                                                  
▌     • Turkish Airlines/LATAM (codeshare)                                                                       
▌     • Número do Voo: TK... / LA... (números a serem confirmados no momento da reserva)                         
▌     • Horário de Partida de IST: A definir                                                                     
▌     • Chegada para conexão (provavelmente em uma cidade europeia com voo codeshare para GRU).                  
▌     • Partida para GRU (São Paulo).                                                                            
▌     • Horário de Chegada em GRU: A definir                                                                     
▌ Observação: Os horários e números de voo acima são baseados em informações gerais sobre rotas codeshare entre  
▌ LATAM e Turkish Airlines e podem sofrer alterações. É fundamental consultar o site oficial da LATAM ou entrar  
▌ em contato para obter a confirmação exata dos voos disponíveis e seus detalhes para as datas desejadas.        

----------------------------------------------------------------

 Algumas dicas de Hoteis, refeiçoes e passeios turisiticos 



▌ Para sua viagem de mochileiro a Istambul entre 15/06/2025 e 30/06/2025, aqui estão 3 opções resumidas de       
▌ hotéis, refeições e passeios turísticos, focando em custo-benefício e experiências autênticas:                 
▌ Hotéis (Hostels com bom custo-benefício):                                                                      
▌  1 Cheers Hostel: Frequentemente recomendado para viajantes individuais e com ótima localização no bairro      
▌    histórico de Sultanahmet, próximo a atrações principais. Oferece terraço com vista e atividades sociais.    
▌  2 Nobel Hostel: Localizado no coração da Cidade Velha (Sultanahmet), também perto da Mesquita Azul. Possui    
▌    quartos privativos e dormitórios, alguns com vista para a Mesquita Azul a partir do terraço.                
▌  3 Istanbul Paris Hotel & Hostel: Uma opção acessível com boa avaliação, também voltado para o público jovem e 
▌    viajantes individuais.                                                                                      
▌ Refeições (Opções Econômicas e Locais):                                                                        
▌  1 Esnaf Lokantası: Procure por esses restaurantes tipo cantina frequentados por trabalhadores locais. Eles    
▌    oferecem comida caseira turca deliciosa e a preços acessíveis.                                              
▌  2 Street Food (Comida de Rua): Experimente o "Balık Ekmek" (sanduíche de peixe) perto da Ponte de Gálata,     
▌    "Döner" e "Lahmacun" (uma espécie de pizza turca fina). Simit (pão em forma de anel com gergelim) é uma     
▌    opção barata para o café da manhã.                                                                          
▌  3 Büfes: Pequenas lojas de conveniência que vendem comida barata como döner, sopa de lentilha ("mercimek      
▌    çorbası") e sanduíches tostados ("tost").                                                                   
▌ Passeios Turísticos (Foco em Experiências Econômicas ou Gratuitas):                                            
▌  1 Explorar Sultanahmet a Pé: Visite a área histórica de Sultanahmet, admirando a Hagia Sophia (por fora, a    
▌    entrada pode ter custo), a Mesquita Azul (entrada gratuita fora dos horários de oração) e o Hipódromo.      
▌    Passear pela praça é gratuito.                                                                              
▌  2 Grand Bazaar e Spice Bazaar: Perca-se nos coloridos e movimentados mercados. A entrada é gratuita e a       
▌    atmosfera é uma atração por si só.                                                                          
▌  3 Passeio de Ferry Público: Atravesse o Bósforo em um ferry público para ter vistas incríveis da cidade e     
▌    conectar-se entre os lados Europeu e Asiático. É uma forma muito econômica de ver o estreito.               
▌ Lembre-se de obter um Istanbulkart para usar o transporte público de forma econômica (metrô, tram, ônibus e    
▌ ferries municipais).                                                                                           

----------------------------------------------------------------
